In [1]:
import argparse
import configparser
import os
from os.path import join, exists, isfile
from os import makedirs

from tqdm.auto import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np

from patchnetvlad.tools.datasets import PlaceDataset
from patchnetvlad.models.models_generic import get_backend, get_model, get_pca_encoding
from patchnetvlad.tools import PATCHNETVLAD_ROOT_DIR

In [2]:
config = configparser.ConfigParser()
config.read(join(PATCHNETVLAD_ROOT_DIR, 'configs/performance.ini'))

['/home/dchan/workspace/Access/Depth/DIFFNet/patchnetvlad/configs/performance.ini']

In [3]:
encoder_dim, encoder = get_backend()
resume_ckpt='/home/dchan/workspace/Access/Depth/Patch-NetVLAD/patchnetvlad/./pretrained_models/mapillary_WPCA4096.pth.tar'
checkpoint = torch.load(resume_ckpt, map_location=lambda storage, loc: storage)
config['global_params']['num_clusters'] = str(checkpoint['state_dict']['pool.centroids'].shape[0])
pool_size = int(config['global_params']['num_pcs'])

In [4]:
model = get_model(encoder, encoder_dim, config['global_params'], append_pca_layer=True)

In [5]:
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [6]:
input_data=torch.randn((1,3,256,256))
image_encoding = model.encoder(input_data)
vlad_local, vlad_global = model.pool(image_encoding)
vlad_global_pca = get_pca_encoding(model, vlad_global)
for this_iter, this_local in enumerate(vlad_local):
    this_patch_size = model.pool.patch_sizes[this_iter]
    db_feat_patches = np.empty((this_local.size(0), pool_size, this_local.size(2)), dtype=np.float32)
    grid = np.indices((1, this_local.size(0)))
    this_local_pca = get_pca_encoding(model, this_local.permute(2, 0, 1).reshape(-1, this_local.size(1))).\
        reshape(this_local.size(2), this_local.size(0), pool_size).permute(1, 2, 0)
    db_feat_patches[grid, :, :] = this_local_pca.detach().cpu().numpy()

In [7]:
from Load_patchNet import load_patchNet

In [8]:
load_patchNet()

Module(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation